In [ ]:
import torch
import treescope
from models.gan import DinoPatchDiscriminator
from torchmetrics.image import (
    StructuralSimilarityIndexMeasure,
    PeakSignalNoiseRatio,
    FrechetInceptionDistance,
    LearnedPerceptualImagePatchSimilarity,
)

treescope.register_as_default()

In [ ]:
model = DinoPatchDiscriminator('small')
model(torch.randn(1, 3, 256, 256).cuda())

In [ ]:
from layers.layers2d import Encoder, Decoder
from models.vae import VAE

In [ ]:
vae = VAE(
    in_channels=3,
    out_channels=3,
    channels=128,
    channels_mult=[1, 2, 4, 4],
    num_res_blocks=2,
    attn_resolutions=[32],
    dropout=0.0,
    resolution=256,
    z_channels=4,
    spatial_compression=4,
    wavelet="db4",
    maxlevel=2
)
vae

In [ ]:
vae.forward(torch.randn(1, 3, 256, 256))

In [ ]:
encoder = Encoder(
    in_channels=3,
    channels=128,
    channels_mult=[1, 2, 4, 4],
    num_res_blocks=2,
    attn_resolutions=[32],
    resolution=256,
    z_channels=4,
    spatial_compression=8,
    dropout=0.1,
)

decoder = Decoder(
    z_channels=4,
    channels=128,
    channels_mult=[1, 2, 4, 4],
    num_res_blocks=2,
    attn_resolutions=[32],
    resolution=256,
    out_channels=3,
    spatial_compression=8,
    dropout=0.1,
)

In [ ]:
encoder

In [ ]:
decoder

In [ ]:
decoder.forward(torch.randn(1, 4, 8, 8)).shape

In [1]:
import torch
import einops
import treescope

from models.vqvae import VQVAE
from layers.quantizers import FSQuantizer
treescope.register_as_default()
treescope.basic_interactive_setup(autovisualize_arrays=True)


In [2]:
a = torch.randn((1, 1, 8))
b = torch.randn((1, 2, 8))

print(einops.rearrange(a, "... c d -> ... (c d)").shape)
print(einops.rearrange(b, "... c d -> ... (c d)").shape)

torch.Size([1, 8])
torch.Size([1, 16])


In [ ]:
quantizer = FSQuantizer(levels=[7, 5, 5, 5], input_dim=256, num_codebooks=1)

In [ ]:
codes, indices = quantizer(torch.randn(1, 256, 8, 8))

In [ ]:
proj_codes, codes = quantizer.indices_to_codes(indices)

In [2]:
vq = VQVAE(
    in_channels=3,
    out_channels=3,
    channels=128,
    channels_mult=[1, 2, 4, 4],
    num_res_blocks=2,
    attn_resolutions=[32],
    dropout=0.0,
    resolution=256,
    z_channels=256,
    spatial_compression=4,
    wavelet="db4",
    maxlevel=2,
    quantization="fsq",
    levels=[8, 5, 5, 5],
    embedding_dim=4,
    num_codebooks=1,
)

Wavelet Transform: db4
z of shape: (1, 256, 16, 16), dimensions: 65536


In [4]:
vq

VQVAE(
  (wavelet_transform): WaveletTransform()
  (encoder): Encoder(
    (conv_in): Conv2d(48, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (down): ModuleList(
      (0): Module(
        (block): ModuleList(
          (0-1): 2 x ResnetBlock(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm2): GroupNorm(32, 128, eps=1e-06, affine=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (nin_shortcut): Identity()
          )
        )
        (attn): ModuleList()
        (downsample): Downsample(
          (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2))
        )
      )
      (1): Module(
        (block): ModuleList(
          (0): ResnetBlock(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (conv1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm2): GroupNorm(32, 256, eps=1e-06, affine=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (nin_shortcut): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1))
          )
          (1): ResnetBlock(
            (norm1): GroupNorm(32, 256, eps=1e-06, affine=True)
            (conv1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm2): GroupNorm(32, 256, eps=1e-06, affine=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (nin_shortcut): Identity()
          )
        )
        (attn): ModuleList(
          (0-1): 2 x AttnBlock(
            (norm): GroupNorm(32, 256, eps=1e-06, affine=True)
            (qkv): Conv2d(256, 768, kernel_size=(1, 1), stride=(1, 1))
            (proj_out): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
          )
        )
        (downsample): Downsample(
          (conv): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2))
        )
      )
      (2): Module(
        (block): ModuleList(
          (0): ResnetBlock(
            (norm1): GroupNorm(32, 256, eps=1e-06, affine=True)
            (conv1): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm2): GroupNorm(32, 512, eps=1e-06, affine=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (nin_shortcut): Conv2d(256, 512, kernel_size=(1, 1), stride=(1, 1))
          )
          (1): ResnetBlock(
            (norm1): GroupNorm(32, 512, eps=1e-06, affine=True)
            (conv1): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm2): GroupNorm(32, 512, eps=1e-06, affine=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (nin_shortcut): Identity()
          )
        )
        (attn): ModuleList()
      )
      (3): Module(
        (block): ModuleList(
          (0-1): 2 x ResnetBlock(
            (norm1): GroupNorm(32, 512, eps=1e-06, affine=True)
            (conv1): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm2): GroupNorm(32, 512, eps=1e-06, affine=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (nin_shortcut): Identity()
          )
        )
        (attn): ModuleList()
      )
    )
    (mid): Module(
      (block1): ResnetBlock(
        (norm1): GroupNorm(32, 512, eps=1e-06, affine=True)
        (conv1): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (norm2): GroupNorm(32, 

In [3]:
vq(torch.randn(1, 3, 256, 256))

(tensor([[[[-0.0061, -0.0140, -0.0634,  ...,  0.0524, -0.0173,  0.0006],
           [-0.0088, -0.0338, -0.1186,  ...,  0.0519, -0.0031, -0.0118],
           [-0.0013, -0.0554, -0.2006,  ...,  0.0338,  0.0260, -0.0306],
           ...,
           [-0.0074,  0.0181,  0.0613,  ...,  0.0022,  0.0120, -0.0112],
           [ 0.0203,  0.0247,  0.0316,  ...,  0.0685, -0.0262, -0.0018],
           [-0.0159, -0.0285, -0.0493,  ..., -0.0539,  0.0195,  0.0026]],
 
          [[-0.0948, -0.1167, -0.1313,  ..., -0.0283,  0.0147, -0.0017],
           [-0.1008, -0.1204, -0.1410,  ..., -0.0956,  0.0024,  0.0312],
           [-0.0931, -0.1056, -0.1322,  ..., -0.2209, -0.0221,  0.0936],
           ...,
           [-0.0581, -0.0273,  0.0692,  ...,  0.0803,  0.1717, -0.1730],
           [-0.0832, -0.0677, -0.0324,  ...,  0.0378, -0.0279,  0.0118],
           [ 0.0913,  0.0681,  0.0057,  ..., -0.0574, -0.0318,  0.0456]],
 
          [[-0.0779, -0.0878, -0.1150,  ...,  0.0049, -0.0120,  0.0088],
           [-0.0760, -0.0873, -0.1162,  ..., -0.0144, -0.0147,  0.0172],
           [-0.0665, -0.0829, -0.1195,  ..., -0.0764, -0.0201,  0.0415],
           ...,
           [ 0.0908,  0.0903,  0.1190,  ...,  0.0606, -0.0328,  0.0058],
           [ 0.1172,  0.0956,  0.0749,  ...,  0.1271, -0.0139, -0.0328],
           [-0.1293, -0.1095, -0.0997,  ..., -0.1286,  0.0233,  0.0256]]]],
        grad_fn=<AddBackward0>),
 (tensor([[[[500],
            [500],
            [500],
            [500],
            [492],
            [500],
            [540],
            [492],
            [500],
            [500],
            [500],
            [500],
            [500],
            [500],
            [500],
            [500]],
  
           [[501],
            [500],
            [499],
            [500],
            [499],
            [693],
            [499],
            [500],
            [500],
            [500],
            [500],
            [500],
            [500],
            [540],
            [500],
            [499]],
  
           [[500],
            [492],
            [739],
            [499],
            [499],
            [532],
            [501],
            [499],
            [741],
            [499],
            [491],
            [691],
            [492],
            [740],
            [540],
            [500]],
  
           [[500],
            [499],
            [699],
            [500],
            [540],
            [539],
            [500],
            [691],
            [540],
            [499],
            [699],
            [699],
            [491],
            [492],
            [500],
            [540]],
  
           [[500],
            [500],
            [499],
            [492],
            [492],
            [499],
            [500],
            [692],
            [500],
            [500],
            [499],
            [732],
            [532],
            [492],
            [700],
            [500]],
  
           [[500],
            [492],
            [540],
            [692],
            [701],
            [532],
            [491],
            [493],
            [499],
            [501],
            [699],
            [700],
            [492],
            [500],
            [741],
            [500]],
  
           [[500],
            [500],
            [492],
            [692],
            [740],
            [492],
            [740],
            [499],
            [501],
            [499],
            [493],
            [732],
            [532],
            [501],
            [541],
            [501]],
  
           [[500],
            [500],
            [700],
            [501],
            [492],
            [500],
            [500],
            [700],
            [501],
            [460],
            [539],
            [500],
            [492],
            [500],
            [500],
            [499]],
  
           [[500],
            [492],
            [491],
            [541],
            [500],
            [492],
            [500],